In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from datetime import datetime
from datetime import timedelta
% matplotlib inline
sns.set(style="darkgrid")

In [2]:
from bs4 import BeautifulSoup
import requests
import pprint
import re

In [19]:
times1 = datetime.now()
df = pd.read_csv('flight_data.csv')
times2 = datetime.now()
print('Time spent: '+ str(times2-times1)) # print the time cost

C:\Users\Qingyu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (48,77,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Time spent: 0:02:03.253791


## Extract tail numbers, keep the unique values

In [20]:
tail_num = pd.Series(df['TAIL_NUM'].unique()).dropna()
tail_num_df = pd.DataFrame()
# create a new DataFrame for tail num 
tail_num_df['tail_num'] = tail_num
tail_num = list(tail_num)

In [23]:
tail_num

['N3754A',
 'N923DL',
 'N551NW',
 'N334NW',
 'N675DL',
 'N309US',
 'N940DL',
 'N920DL',
 'N357NW',
 'N199DN',
 'N943AT',
 'N3733Z',
 'N925AT',
 'N973DL',
 'N907DE',
 'N905DA',
 'N392DA',
 'N927DN',
 'N302NB',
 'N966DL',
 'N341NW',
 'N970AT',
 'N328NW',
 'N608AT',
 'N913DL',
 'N378NW',
 'N703TW',
 'N927DA',
 'N752AT',
 'N915DL',
 'N955DN',
 'N842DN',
 'N538US',
 'N352NW',
 'N681DA',
 'N6716C',
 'N965DL',
 'N976DL',
 'N951DN',
 'N683DA',
 'N546US',
 'N370NB',
 'N991AT',
 'N347NW',
 'N910DL',
 'N971AT',
 'N981DL',
 'N603AT',
 'N362NW',
 'N959AT',
 'N338NW',
 'N343NB',
 'N584NW',
 'N924AT',
 'N3739P',
 'N3757D',
 'N377DA',
 'N358NB',
 'N67171',
 'N916DE',
 'N982AT',
 'N909DE',
 'N819DN',
 'N989AT',
 'N906DA',
 'N6701',
 'N668DN',
 'N910AT',
 'N958DL',
 'N391DA',
 'N933AT',
 'N904DE',
 'N913DE',
 'N1611B',
 'N303DQ',
 'N994AT',
 'N824DN',
 'N956DL',
 'N906DL',
 'N333NB',
 'N964DL',
 'N896AT',
 'N907DL',
 'N971DL',
 'N315US',
 'N321NB',
 'N910DN',
 'N904DL',
 'N827DN',
 'N920AT',
 'N812DN',


### The url is 'https://registry.faa.gov/aircraftinquiry/NNum_Results.aspx?NNumbertxt=NXXXXX'

In [ ]:
mfr_name = []
model = []
year = []
eng_mfr_name = []
eng_model = []
aircraft_type = []

In [ ]:
times1 = datetime.now()
for url in tail_num:
    if isinstance(url, str):
        response = requests.get(
            'https://registry.faa.gov/aircraftinquiry/NNum_Results.aspx?NNumbertxt=' + url
        )
        soup = BeautifulSoup(response.content, "html.parser")
        # determine if the planes has retired    
        if soup.find('span', id='content_lbMfrName') or soup.find('span', id='content_Label7') or soup.find('span', id='content_Label17'):
            # determine if there is a MFR name
            if soup.find('span', id='content_lbMfrName').string != 'None':
                mfr_name.append(soup.find('span', id='content_lbMfrName').string.strip())
            else:
                mfr_name.append(np.nan)
            # determine if there is a model
            if soup.find('span', id='content_Label7').string != 'None':
                model.append(soup.find('span', id='content_Label7').string.strip())
            else:
                model.append(np.nan)
            # determine if there is a MFR year
            if soup.find('span', id='content_Label17').string != 'None':
                year.append(int(soup.find('span', id='content_Label17').string.strip()))
            # use the year in Certificate Issue Date
            elif soup.find('span', id='content_lbCertDate').string != 'None':
                year.append(int(re.search(
                    r'\w+/\w+/(\w+)', soup.find('span', id='content_lbCertDate').string
                ).group(1).strip()))
            else:
                 year.append(np.nan)
            # determine if there is an eng mfr name
            if soup.find('span', id='content_lbEngMfr').string != 'None':
                eng_mfr_name.append(soup.find('span', id='content_lbEngMfr').string.strip())
            else:
                eng_mfr_name.append(np.nan)
            # determine if there is an eng model
            if soup.find('span', id='content_lbEngModel').string != 'None':
                eng_model.append(soup.find('span', id='content_lbEngModel').string.strip())
            else:
                eng_model.append(np.nan)
            # determine if there is a type
            if soup.find('span', id='content_Label11').string != 'None':
                aircraft_type.append(soup.find('span', id='content_Label11').string.strip())
            else:
                aircraft_type.append(np.nan)
        # the plane has retired
        elif soup.find('span', id='content_drptrDeRegAircraft_lbDeRegMfrName_0') or soup.find('span', id='content_drptrDeRegAircraft_lbDeRegModel_0') or soup.find('span', id='content_drptrDeRegAircraft_lbDeRegYearMfr_0'): 
            if soup.find('span', id='content_drptrDeRegAircraft_lbDeRegMfrName_0').string != 'None':
                mfr_name.append(soup.find('span', id='content_drptrDeRegAircraft_lbDeRegMfrName_0').string.strip())
            else:
                mfr_name.append(np.nan)
            # determine if there is a model
            if soup.find('span', id='content_drptrDeRegAircraft_lbDeRegModel_0').string != 'None':
                model.append(soup.find('span', id='content_drptrDeRegAircraft_lbDeRegModel_0').string.strip())
            else:
                model.append(np.nan)
            # determine if there is a MFR year
            if soup.find('span', id='content_drptrDeRegAircraft_lbDeRegYearMfr_0').string != 'None':
                year.append(int(soup.find('span', id='content_drptrDeRegAircraft_lbDeRegYearMfr_0').string.strip()))
            # use the year in Certificate Issue Date
            elif soup.find('span', id='content_drptrDeRegAircraft_lbDeRegCertDate_0').string != 'None':
                year.append(int(re.search(
                    r'\w+/\w+/(\w+)', soup.find('span', id='content_drptrDeRegAircraft_lbDeRegCertDate_0').string
                ).group(1).strip()))
            else:
                 year.append(np.nan)
            # determine if there is an eng mfr name
            if soup.find('span', id='content_drptrDeRegAircraft_lbDREngMfr_0').string != 'None':
                eng_mfr_name.append(soup.find('span', id='content_drptrDeRegAircraft_lbDREngMfr_0').string.strip())
            else:
                eng_mfr_name.append(np.nan)
            # determine if there is an eng model
            if soup.find('span', id='content_drptrDeRegAircraft_lbDREngModel_0').string != 'None':
                eng_model.append(soup.find('span', id='content_drptrDeRegAircraft_lbDREngModel_0').string.strip())
            else:
                eng_model.append(np.nan)
            aircraft_type.append(np.nan)
        else:
            mfr_name.append(np.nan)
            model.append(np.nan)
            year.append(np.nan)
            eng_mfr_name.append(np.nan)
            eng_model.append(np.nan)
            aircraft_type.append(np.nan)
    else:
        mfr_name.append(np.nan)
        model.append(np.nan)
        year.append(np.nan)
        eng_mfr_name.append(np.nan)
        eng_model.append(np.nan)
        aircraft_type.append(np.nan)
tail_num_df['mfr_name'] = mfr_name
tail_num_df['model'] = model
tail_num_df['year'] = year
tail_num_df['eng_mfr_name'] = eng_mfr_name
tail_num_df['eng_model'] = eng_model
tail_num_df['aircraft_type'] = aircraft_type
times2 = datetime.now()
print('Time spent: '+ str(times2-times1)) # print the time cost
tail_num_df.to_csv('./tail_num/tail_num.csv', index = False)

In [ ]:
tail_num_df

In [ ]:
len(eng_model)

In [ ]:
tail_num[905] == np.nan

In [ ]:
isinstance(url, str)

In [ ]:
2018-2002.0

In [ ]:
tail_num_df.to_csv('tail_num.csv', index = False)

In [ ]:
if soup.find('span', id='content_lbMfrName'):
    pass
else:
    mfr = soup.find('span', id='content_drptrDeRegAircraft_lbDeRegMfrName_0')
mfr

In [ ]:
# s = soup.find('span', id='content_lbCertDate').string
# s.replace('\\','/')
# re.search(r'\w+/\w+/(\w+)', s).group(1)
int(re.search(r'\w+/\w+/(\w+)', soup.find('span', id='content_lbCertDate').string).group(1).strip())

In [ ]:
mfr_name.append(soup.find('span', id='content_lbMfrName').string.strip())
model.append(soup.find('span', id='content_Label7').string.strip())
year.append(int(soup.find('span', id='content_Label17').string.strip()))
eng_mfr_name.append(soup.find('span', id='content_lbEngMfr').string.strip())
eng_model.append(soup.find('span', id='content_lbEngModel').string.strip())

In [ ]:
t_num = ['N3754A', 'N752AT']
df_t_num = pd.DataFrame()
df_t_num['tail_num'] = t_num

In [ ]:
df_t_num['mfr_name'] = mfr_name
df_t_num['model'] = model

In [ ]:
df_t_num

In [ ]:
mfr_name = []
model = []
year = []
eng_mfr_name = []
eng_model = []
t_num = ['N3754A', 'N923DL']
df_t_num = pd.DataFrame()
df_t_num['tail_num'] = t_num
for url in t_num:
    response = requests.get('https://registry.faa.gov/aircraftinquiry/NNum_Results.aspx?NNumbertxt='+url)
    soup = BeautifulSoup(response.content, "html.parser")
    mfr_name.append(soup.find('span', id='content_lbMfrName').string.strip())
    model.append(soup.find('span', id='content_Label7').string.strip())
    year.append(int(soup.find('span', id='content_Label17').string.strip()))
    eng_mfr_name.append(soup.find('span', id='content_lbEngMfr').string.strip())
    eng_model.append(soup.find('span', id='content_lbEngModel').string.strip())
df_t_num['mfr_name'] = mfr_name
df_t_num['model'] = model
df_t_num['year'] = year
df_t_num['eng_mfr_name'] = eng_mfr_name
df_t_num['eng_model'] = eng_model

In [ ]:
df_t_num

### <td><span class="Results_DataText" id="content_lbMfrName">BOEING                        </span></td>
### <td><span class="Results_DataText" id="content_Label7">737-832             </span></td>
### <td><span class="Results_DataText" id="content_Label17">2001</span></td>
### <td style="width:30%"><span class="Results_DataText" id="content_lbEngMfr">CFM INTL. </span></td>
### <td><span class="Results_DataText" id="content_lbEngModel">CFM56 SERIES </span></td>
### <td><span class="Results_DataText" id="content_lbCertDate">08/29/2002</span></td>
### deresgistred year <td class="boxed-LightCyan"><span class="Results_DataText" id="content_drptrDeRegAircraft_lbDeRegYearMfr_0">1984</span></td>
### deresgistred cert issue date <td class="boxed-LightCyan"><span class="Results_DataText" id="content_drptrDeRegAircraft_lbDeRegCertDate_0">11/04/1996</span></td></tr>

## N442CA 2 deregistered

In [ ]:
df = pd.read_csv('./tail_num/tail_num.csv')

In [ ]:
idx = list(np.array(df.iloc[:, 1:5].dropna(how='all').index))
df = df.iloc[idx]

In [ ]:
df_missing = df[df.iloc[:, 1:5].isnull().all(axis=1)]
df_missing = df_missing.dropna(how='all')

In [ ]:
df_missing.loc[819, 'tail_num'] = 'N531JL'

In [ ]:
old_tail = []
new_tail = []

In [ ]:
old_tail.append('N852AA')
new_tail.append('N120EE')

In [ ]:
new_tail

In [ ]:
df_conv_tail_num = pd.DataFrame()
df_conv_tail_num['old_tail_num'] = df_missing['tail_num']

In [ ]:
df_conv_tail_num['new_tail_num'] = np.nan

In [ ]:
df_conv_tail_num = df_conv_tail_num.reset_index(drop=True)

In [ ]:
aa_df = pd.read_csv('./tail_num/aa_current_fleet_tail_num_convert.csv')

In [ ]:
# match AA current fleet
for i in range(df_conv_tail_num.shape[0]):
    for j in range(aa_df.shape[0]):
        if df_conv_tail_num.loc[i, 'old_tail_num'] == aa_df.loc[j, 'old_tail_num']:
            df_conv_tail_num.loc[i, 'new_tail_num'] = aa_df.loc[j, 'new_tail_num']
            break

In [ ]:
aa_ret_df = pd.read_csv('./tail_num/aa_retired_fleet_tail_num_convert.csv')

In [ ]:
# match AA retire fleet
for i in range(df_conv_tail_num.shape[0]):
    if not isinstance(df_conv_tail_num.loc[i, 'new_tail_num'], str):
        for j in range(aa_ret_df.shape[0]):
            if df_conv_tail_num.loc[i, 'old_tail_num'] == aa_ret_df.loc[j, 'old_tail_num']:
                df_conv_tail_num.loc[i, 'new_tail_num'] = aa_ret_df.loc[j, 'new_tail_num']
                break

In [ ]:
# match AA current fleet ignore US/AA in the end
for i in range(df_conv_tail_num.shape[0]):
    if not isinstance(df_conv_tail_num.loc[i, 'new_tail_num'], str):
        for j in range(aa_df.shape[0]):
            if df_conv_tail_num.loc[i, 'old_tail_num'][:-2] == aa_df.loc[j, 'old_tail_num'][:-2]:
                df_conv_tail_num.loc[i, 'new_tail_num'] = aa_df.loc[j, 'new_tail_num']
                break

In [ ]:
df_conv_tail_num

In [ ]:
df_conv_tail_num.to_csv('./tail_num/tail_num_convert.csv', index = False)

In [ ]:
df_conv_tail_num[df_conv_tail_num['old_tail_num'] == 'N7820J']

In [ ]:
df_conv_tail_num.loc[476, 'new_tail_num'] = 'N7820L'

In [ ]:
df.groupby('mfr_name').describe().year.sort_values(by=['count'], ascending=False)

In [ ]:
df[df['mfr_name'] == 'DJI']

In [ ]:
times1 = datetime.now()
df_flight = pd.read_csv('flight_data.csv')
times2 = datetime.now()
print('Time spent: '+ str(times2-times1)) # print the time cost

In [ ]:
# df_flight[(df_flight['TAIL_NUM'] == 'N498SW') & (df_flight['MONTH'] > 10)].iloc[:,:11]
df_flight[df_flight['TAIL_NUM'] == 'N498SW'].iloc[:,:11]

# scrape data for matched tail numbers

In [ ]:
mfr_name = []
model = []
year = []
eng_mfr_name = []
eng_model = []
aircraft_type = []

In [ ]:
tail_num = pd.read_csv('./tail_num/tail_num_convert.csv')
tail_num_lst = tail_num['new_tail_num'].tolist()

In [ ]:
tail_num_df = pd.DataFrame()
tail_num_df['tail_num'] = tail_num['old_tail_num']

In [ ]:
times1 = datetime.now()
for url in tail_num_lst:
    if isinstance(url, str):
        response = requests.get(
            'https://registry.faa.gov/aircraftinquiry/NNum_Results.aspx?NNumbertxt=' + url
        )
        soup = BeautifulSoup(response.content, "html.parser")
        # determine if the planes has retired    
        if soup.find('span', id='content_lbMfrName') or soup.find('span', id='content_Label7') or soup.find('span', id='content_Label17'):
            # determine if there is a MFR name
            if soup.find('span', id='content_lbMfrName').string != 'None':
                mfr_name.append(soup.find('span', id='content_lbMfrName').string.strip())
            else:
                mfr_name.append(np.nan)
            # determine if there is a model
            if soup.find('span', id='content_Label7').string != 'None':
                model.append(soup.find('span', id='content_Label7').string.strip())
            else:
                model.append(np.nan)
            # determine if there is a MFR year
            if soup.find('span', id='content_Label17').string != 'None':
                year.append(int(soup.find('span', id='content_Label17').string.strip()))
            # use the year in Certificate Issue Date
            elif soup.find('span', id='content_lbCertDate').string != 'None':
                year.append(int(re.search(
                    r'\w+/\w+/(\w+)', soup.find('span', id='content_lbCertDate').string
                ).group(1).strip()))
            else:
                 year.append(np.nan)
            # determine if there is an eng mfr name
            if soup.find('span', id='content_lbEngMfr').string != 'None':
                eng_mfr_name.append(soup.find('span', id='content_lbEngMfr').string.strip())
            else:
                eng_mfr_name.append(np.nan)
            # determine if there is an eng model
            if soup.find('span', id='content_lbEngModel').string != 'None':
                eng_model.append(soup.find('span', id='content_lbEngModel').string.strip())
            else:
                eng_model.append(np.nan)
            
            # determine if there is a type
            if soup.find('span', id='content_Label11').string != 'None':
                aircraft_type.append(soup.find('span', id='content_Label11').string.strip())
            else:
                aircraft_type.append(np.nan)
            
                
                
        # the plane has retired
        elif soup.find('span', id='content_drptrDeRegAircraft_lbDeRegMfrName_0') or soup.find('span', id='content_drptrDeRegAircraft_lbDeRegModel_0') or soup.find('span', id='content_drptrDeRegAircraft_lbDeRegYearMfr_0'): 
            if soup.find('span', id='content_drptrDeRegAircraft_lbDeRegMfrName_0').string != 'None':
                mfr_name.append(soup.find('span', id='content_drptrDeRegAircraft_lbDeRegMfrName_0').string.strip())
            else:
                mfr_name.append(np.nan)
            # determine if there is a model
            if soup.find('span', id='content_drptrDeRegAircraft_lbDeRegModel_0').string != 'None':
                model.append(soup.find('span', id='content_drptrDeRegAircraft_lbDeRegModel_0').string.strip())
            else:
                model.append(np.nan)
            # determine if there is a MFR year
            if soup.find('span', id='content_drptrDeRegAircraft_lbDeRegYearMfr_0').string != 'None':
                year.append(int(soup.find('span', id='content_drptrDeRegAircraft_lbDeRegYearMfr_0').string.strip()))
            # use the year in Certificate Issue Date
            elif soup.find('span', id='content_drptrDeRegAircraft_lbDeRegCertDate_0').string != 'None':
                year.append(int(re.search(
                    r'\w+/\w+/(\w+)', soup.find('span', id='content_drptrDeRegAircraft_lbDeRegCertDate_0').string
                ).group(1).strip()))
            else:
                 year.append(np.nan)
            # determine if there is an eng mfr name
            if soup.find('span', id='content_drptrDeRegAircraft_lbDREngMfr_0').string != 'None':
                eng_mfr_name.append(soup.find('span', id='content_drptrDeRegAircraft_lbDREngMfr_0').string.strip())
            else:
                eng_mfr_name.append(np.nan)
            # determine if there is an eng model
            if soup.find('span', id='content_drptrDeRegAircraft_lbDREngModel_0').string != 'None':
                eng_model.append(soup.find('span', id='content_drptrDeRegAircraft_lbDREngModel_0').string.strip())
            else:
                eng_model.append(np.nan)
            aircraft_type.append(np.nan)
        else:
            mfr_name.append(np.nan)
            model.append(np.nan)
            year.append(np.nan)
            eng_mfr_name.append(np.nan)
            eng_model.append(np.nan)
            aircraft_type.append(np.nan)
    else:
        mfr_name.append(np.nan)
        model.append(np.nan)
        year.append(np.nan)
        eng_mfr_name.append(np.nan)
        eng_model.append(np.nan)
        aircraft_type.append(np.nan)
tail_num_df['mfr_name'] = mfr_name
tail_num_df['model'] = model
tail_num_df['year'] = year
tail_num_df['eng_mfr_name'] = eng_mfr_name
tail_num_df['eng_model'] = eng_model
tail_num_df['aircraft_type'] = aircraft_type
times2 = datetime.now()
print('Time spent: '+ str(times2-times1)) # print the time cost
tail_num_df.to_csv('./tail_num/matched_tail_num.csv', index = False)

In [ ]:
tail_num_df_1 = tail_num_df

In [ ]:
tail_num_df = pd.read_csv('./tail_num/matched_tail_num.csv')

In [ ]:
idx = tail_num_df_1.iloc[:, 1:5].dropna(how='all').index.tolist()

In [ ]:
tail_num_df_1 = tail_num_df_1.loc[idx]

In [ ]:
tail_num_df_1 = pd.concat([tail_num_df_1, tail_num_df])

In [ ]:
tail_num_df_1.reset_index(drop=True)
tail_num_df_1.to_csv('./tail_num/all_tail_num.csv', index=False)

In [24]:
df = pd.read_csv('./tail_num/all_tail_num.csv')

In [ ]:
df_missing = df[df.iloc[:, 1:5].isnull().all(axis=1)]

In [ ]:
df_missing

In [ ]:
df.groupby('aircraft_type').describe()

In [ ]:
df[df['aircraft_type'] == 'Rotorcraft']

In [34]:
df[df['tail_num'].str.match(r'^N\w\w\wAA$')]

tail_num                   mfr_name           model    year eng_mfr_name  \
3583   N434AA          MCDONNELL DOUGLAS  DC-9-83(MD-83)  1987.0        P & W   
3584   N489AA          MCDONNELL DOUGLAS  DC-9-82(MD-82)  1989.0        P & W   
3585   N439AA          MCDONNELL DOUGLAS  DC-9-83(MD-83)  1987.0        P & W   
3586   N468AA          MCDONNELL DOUGLAS  DC-9-82(MD-82)  1988.0        P & W   
3587   N569AA          MCDONNELL DOUGLAS  DC-9-83(MD-83)  1987.0        P & W   
3588   N466AA          MCDONNELL DOUGLAS  DC-9-82(MD-82)  1988.0        P & W   
3589   N501AA          MCDONNELL DOUGLAS  DC-9-82(MD-82)  1989.0        P & W   
3590   N438AA          MCDONNELL DOUGLAS  DC-9-83(MD-83)  1987.0        P & W   
3591   N426AA          MCDONNELL DOUGLAS  DC-9-82(MD-82)  1986.0        P & W   
3592   N566AA          MCDONNELL DOUGLAS  DC-9-83(MD-83)  1987.0        P & W   
3593   N499AA          MCDONNELL DOUGLAS  DC-9-82(MD-82)  1989.0        P & W   
3594   N520AA               KILDALL GARY       FALCON-XP  1985.0        ROTAX   
3595   N547AA               STINNETT RON       FALCON XP  1985.0   BOMBARDIER   
3596   N504AA          AUTHIER ANTHONY P       TIERRA II     NaN   BOMBARDIER   
3597   N514AA                        DJI       INSPIRE 2  2018.0      Unknown   
3598   N471AA          MCDONNELL DOUGLAS  DC-9-82(MD-82)  1988.0        P & W   
3599   N565AA          MCDONNELL DOUGLAS  DC-9-83(MD-83)  1987.0        P & W   
3600   N885AA     EXTRA FLUGZEUGBAU GMBH          EA-400  2001.0   CONT MOTOR   
3601   N877AA       TEXTRON AVIATION INC            172S  2018.0     LYCOMING   
3602   N870AA                   SIKORSKY          EH-60A  2018.0      Unknown   
3603   N873AA                     BOEING         727-223  1978.0        P & W   
3604   N883AA                     BOEING         727-223  1979.0          NaN   
3605   N871AA                     BOEING         727-223  1978.0        P & W   
3606   N867AA                     CESSNA            172M  1974.0     LYCOMING   
3607   N882AA  RAYTHEON AIRCRAFT COMPANY             390  2003.0     WILLIAMS   
3608   N874AA                     BOEING         727-223  1978.0        P & W   
3609   N881AA                     BOEING         727-223  1979.0        P & W   
3610   N872AA                     BOEING         727-223  1978.0        P & W   
3611   N866AA                     BOEING         727-223  1977.0        P & W   
3612   N879AA                     BOEING         727-223  1978.0        P & W   
...       ...                        ...             ...     ...          ...   
5314   N3PPAA                     BOEING         737-800  2017.0     CFM INTL   
5315   N3PRAA                     BOEING         737-800  2017.0     CFM INTL   
5316   N3PTAA                     BOEING         737-800  2017.0     CFM INTL   
5317   N3PHAA                     BOEING         737-800  2017.0     CFM INTL   
5318   N3PMAA                     BOEING         737-800  2017.0     CFM INTL   
5319   N3PSAA                     BOEING         737-800  2017.0     CFM INTL   
5320   N3PFAA                     BOEING         737-800  2017.0     CFM INTL   
5321   N3PNAA                     BOEING         737-800  2017.0     CFM INTL   
5322   N3PJAA                     BOEING         737-800  2017.0     CFM INTL   
5323   N3PLAA                     BOEING         737-800  2017.0     CFM INTL   
5324   N3PGAA                     BOEING         737-800  2017.0     CFM INTL   
5325   N3PUAA                     BOEING         737-800  2017.0     CFM INTL   
5326   N8AWAA                     BOEING           787-8  2017.0           GE   
5327   N3PKAA                     BOEING         737-800  2017.0     CFM INTL   
5328   N3PEAA                     BOEING         737-800  2017.0     CFM INTL   
5329   N8LNAA                     BOEING           787-9  2017.0           GE   
5330   N3RAAA                     BOEING           737-8  2017.0     CFM INTL   
5331   N3RBAA                     BO

In [35]:
df[df['tail_num'].str.match(r'^N\w\w\wAA$')].groupby('mfr_name').year.count().sort_values(ascending=False)

mfr_name
BOEING                           549
MCDONNELL DOUGLAS                121
AIRBUS                            62
CESSNA                            56
BEECH                             40
AIRBUS INDUSTRIE                  14
SIKORSKY                          12
PIPER                             11
DASSAULT/SUD                       8
AVIAT AIRCRAFT INC                 7
GULFSTREAM AEROSPACE               7
CIRRUS DESIGN CORP                 6
DJI                                6
AMERICAN AIRCRAFT INC              5
BELL                               5
SHORT BROS                         4
EMBRAER                            4
AERO COMMANDER                     4
DASSAULT-BREGUET                   3
CANADAIR LTD                       3
RAYTHEON AIRCRAFT COMPANY          3
GATES LEAR JET                     3
AGUSTA SPA                         3
DASSAULT AVIATION                  2
TAGUE LARRY                        2
ALPHA M SCIENTIFIC                 2
ASTRAEUS AERIAL LLC          

In [40]:
df_not_aa = df[df['tail_num'].str.match(r'^N\w\w\wAA$').apply(lambda x: not x)]

In [41]:
df_aa = df[df['tail_num'].str.match(r'^N\w\w\wAA$')]

In [43]:
aa_tail_num = df_aa['tail_num']

In [44]:
aa_tail_num

3583    N434AA
3584    N489AA
3585    N439AA
3586    N468AA
3587    N569AA
3588    N466AA
3589    N501AA
3590    N438AA
3591    N426AA
3592    N566AA
3593    N499AA
3594    N520AA
3595    N547AA
3596    N504AA
3597    N514AA
3598    N471AA
3599    N565AA
3600    N885AA
3601    N877AA
3602    N870AA
3603    N873AA
3604    N883AA
3605    N871AA
3606    N867AA
3607    N882AA
3608    N874AA
3609    N881AA
3610    N872AA
3611    N866AA
3612    N879AA
         ...  
5314    N3PPAA
5315    N3PRAA
5316    N3PTAA
5317    N3PHAA
5318    N3PMAA
5319    N3PSAA
5320    N3PFAA
5321    N3PNAA
5322    N3PJAA
5323    N3PLAA
5324    N3PGAA
5325    N3PUAA
5326    N8AWAA
5327    N3PKAA
5328    N3PEAA
5329    N8LNAA
5330    N3RAAA
5331    N3RBAA
5332    N8LJAA
5333    N8LLAA
5334    N8LHAA
5335    N8LKAA
5336    N8LEAA
5337    N8LGAA
5338    N8LMAA
5339    N8AVAA
5340    N3PWAA
5341    N3PVAA
5342    N3PXAA
5343    N8LPAA
Name: tail_num, Length: 1044, dtype: object

In [45]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [46]:
df = pd.read_csv('flight_data.csv')
tail_num = pd.Series(df['TAIL_NUM'].unique()).dropna()
tail_num_df = pd.DataFrame()
# create a new DataFrame for tail num
tail_num_df['tail_num'] = tail_num

C:\Users\Qingyu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (48,77,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [51]:
df_not_aa = tail_num_df[tail_num_df['tail_num'].str.match(r'^N\w\w\wAA$').apply(lambda x: not x)]
df_aa = tail_num_df[tail_num_df['tail_num'].str.match(r'^N\w\w\wAA$')]
df_tail_num_convert = pd.read_csv('./tail_num/tail_num_convert.csv')

In [56]:
df_aa = df_aa.rename(index=str, columns={"tail_num": "old_tail_num"})

In [58]:
df_aa['new_tail_num'] = np.nan

In [59]:
df_aa

old_tail_num  new_tail_num
3597       N4YBAA           NaN
3598       N434AA           NaN
3599       N541AA           NaN
3600       N489AA           NaN
3601       N439AA           NaN
3602       N468AA           NaN
3603       N569AA           NaN
3604       N466AA           NaN
3605       N501AA           NaN
3606       N438AA           NaN
3607       N426AA           NaN
3608       N566AA           NaN
3609       N499AA           NaN
3610       N4YPAA           NaN
3611       N520AA           NaN
3612       N547AA           NaN
3613       N4XBAA           NaN
3614       N504AA           NaN
3615       N4XPAA           NaN
3616       N4UCAA           NaN
3617       N514AA           NaN
3618       N471AA           NaN
3619       N4XVAA           NaN
3620       N4XGAA           NaN
3621       N4YEAA           NaN
3622       N565AA           NaN
3623       N4WJAA           NaN
3624       N4YLAA           NaN
3625       N4XWAA           NaN
3626       N3HMAA           NaN
...           ...           ...
5238       N909AA           NaN
5239       N3PJAA           NaN
5240       N3PLAA           NaN
5241       N933AA           NaN
5242       N3PGAA           NaN
5243       N906AA           NaN
5244       N910AA           NaN
5245       N932AA           NaN
5246       N3PUAA           NaN
5247       N904AA           NaN
5248       N8AWAA           NaN
5249       N3PKAA           NaN
5250       N3PEAA           NaN
5251       N902AA           NaN
5252       N8LNAA           NaN
5253       N3RAAA           NaN
5254       N3RBAA           NaN
5255       N8LJAA           NaN
5256       N8LLAA           NaN
5257       N8LHAA           NaN
5258       N8LKAA           NaN
5259       N8LEAA           NaN
5260       N8LGAA           NaN
5261       N8LMAA           NaN
5262       N8AVAA           NaN
5295       N3PWAA           NaN
5296       N3PVAA           NaN
5297       N3PXAA           NaN
5298       N8LPAA           NaN
5327       N376AA           NaN

[1044 rows x 2 columns]